In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
#import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Specify the folder path
folder_path = "/Users/egkubo/AWID/AWID3_Dataset_CSV/CSV/12.Evil_Twin"

# Use glob to get a list of all CSV files in the folder
csv_files = glob.glob(f"{folder_path}/*.csv")

# Initialize an empty list to store individual DataFrames
dfs = []

# Loop through each CSV file, read it into a DataFrame, and append it to the list
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
data = pd.concat(dfs, ignore_index=True)
data

In [15]:
data_modified = data.loc[:, ['frame.len', 'radiotap.length', 'radiotap.dbm_antsignal', 'wlan.duration', 'radiotap.channel.freq', 'wlan.fc.type', 'wlan.fc.subtype', 'wlan.fc.ds', 'wlan.fc.frag', 'wlan.fc.retry', 'wlan.fc.pwrmgt', 'wlan.fc.moredata', 'wlan.fc.protected', 'Label']]

data_modified = data_modified.replace(r'^\s*$', pd.NA, regex=True)
data_cleaned = data_modified.dropna()

In [30]:
X = data_cleaned.loc[:, ['frame.len', 'radiotap.length', 'radiotap.dbm_antsignal', 'wlan.duration', 'radiotap.channel.freq', 'wlan.fc.type', 'wlan.fc.subtype', 'wlan.fc.ds', 'wlan.fc.frag', 'wlan.fc.retry', 'wlan.fc.pwrmgt', 'wlan.fc.moredata', 'wlan.fc.protected']]
y = data_cleaned[['Label']]
X
# radiotap.present.tsft geri eklemeyi unutma

,frame.len,radiotap.length,radiotap.dbm_antsignal,wlan.duration,radiotap.channel.freq,wlan.fc.type,wlan.fc.subtype,wlan.fc.ds,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected
0,76.0,56.0,-103,158.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
1,70.0,56.0,-198,116.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
2,88.0,56.0,-199,2.0,5180.0,1.0,9.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3,76.0,56.0,-103,226.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
4,70.0,56.0,-209,182.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778701,70.0,56.0,-33-38-33,104.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778702,76.0,56.0,-34-38-34,314.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778703,70.0,56.0,-64-68-64,272.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778704,76.0,56.0,-33-37-33,314.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0


In [33]:
def convert_to_integer(value): # Function to cover the cases where there are multiple dbm_antsignal values.
    if pd.isna(value) or value == '':
        return None 
    elif isinstance(value, int) or isinstance(value, float):
        return value
    else:
        signal_strengths = [int(v) for v in value.split('-') if v]
        if signal_strengths:
            average_strength = round(sum(signal_strengths) / len(signal_strengths))
            return -average_strength
        else:
            return None

In [34]:
X['radiotap.dbm_antsignal'] = X['radiotap.dbm_antsignal'].apply(convert_to_integer)
X

,frame.len,radiotap.length,radiotap.dbm_antsignal,wlan.duration,radiotap.channel.freq,wlan.fc.type,wlan.fc.subtype,wlan.fc.ds,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected
0,76.0,56.0,-103.0,158.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
1,70.0,56.0,-198.0,116.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
2,88.0,56.0,-199.0,2.0,5180.0,1.0,9.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3,76.0,56.0,-103.0,226.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
4,70.0,56.0,-209.0,182.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778701,70.0,56.0,-35.0,104.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778702,76.0,56.0,-35.0,314.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778703,70.0,56.0,-65.0,272.0,5180.0,1.0,12.0,0x00000000,0.0,0.0,0.0,0.0,0.0
3778704,76.0,56.0,-34.0,314.0,5180.0,1.0,11.0,0x00000000,0.0,0.0,0.0,0.0,0.0


In [35]:
# Columns for min-max scaling
columns_to_scale = ['frame.len', 'radiotap.length', 'radiotap.dbm_antsignal', 'wlan.duration']

# Columns for one-hot encoding (excluding the ones to be scaled)
columns_to_one_hot_encode = [col for col in X.columns if col not in columns_to_scale]

# Min-max scaling
scaler = MinMaxScaler()
X_scaled = X.copy()  # Create a copy to avoid modifying the original DataFrame
X_scaled[columns_to_scale] = scaler.fit_transform(X_scaled[columns_to_scale])

# One-hot encoding
X_encoded = pd.get_dummies(X_scaled, columns=columns_to_one_hot_encode)

# Display the preprocessed DataFrame
X_encoded

,frame.len,radiotap.length,radiotap.dbm_antsignal,wlan.duration,radiotap.channel.freq_5180.0,wlan.fc.type_0.0,wlan.fc.type_1.0,wlan.fc.type_2.0,wlan.fc.subtype_0.0,wlan.fc.subtype_1.0,...,wlan.fc.frag_0.0,wlan.fc.frag_1.0,wlan.fc.retry_0.0,wlan.fc.retry_1.0,wlan.fc.pwrmgt_0.0,wlan.fc.pwrmgt_1.0,wlan.fc.moredata_0.0,wlan.fc.moredata_1.0,wlan.fc.protected_0.0,wlan.fc.protected_1.0
0,0.001664,0.5,0.672646,0.005072,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
1,0.000000,0.5,0.246637,0.003724,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
2,0.004992,0.5,0.242152,0.000064,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
3,0.001664,0.5,0.672646,0.007255,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
4,0.000000,0.5,0.197309,0.005842,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778701,0.000000,0.5,0.977578,0.003338,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
3778702,0.001664,0.5,0.977578,0.010080,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
3778703,0.000000,0.5,0.843049,0.008731,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False
3778704,0.001664,0.5,0.982063,0.010080,True,False,True,False,False,False,...,True,False,True,False,True,False,True,False,True,False


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=41)

In [40]:
# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the model
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the model
nb_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {nb_accuracy:.2f}")

# Display additional evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/Users/egkubo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


: 

: 